## 40단계: 브로드캐스트 함수

> .

### 40.1 broadcast_to 함수와 sum_to 함수 (넘파이 버전)

In [1]:
import numpy as np

x = np.array([1, 2, 3])
y = np.broadcast_to(x, (2, 3))
print(y)

[[1 2 3]
 [1 2 3]]


<img src="images/그림 40-1.png" width=500/>


In [2]:
import numpy as np
from dezero.utils import sum_to

x = np.array([[1, 2, 3], [4, 5, 6]])
y = sum_to(x, (1, 3))
print(y)

y = sum_to(x, (2, 1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


<img src="images/그림 40-2.png" width=500/>


### 40.2 broadcast_to 함수와 sum_to 함수 (DeZero 버전)

In [3]:
# dezero/functions.py
from dezero.core import Function, as_variable

class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y

    def backward(self, gy):
        gx = sum_to(gy, self.x_shape)
        return gx


def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)

In [4]:
# dezero.functions.py
from dezero import utils

class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape
        
    def forward(self, x):
        self.x_shape = x.shape
        y = utils.sum_to(x, self.shape)
        return y
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)

### 40.3 브로드캐스트 대응

In [5]:
x0 = np.array([1, 2, 3])
x1 = np.array([10])
y = x0 + x1
print(y)

[11 12 13]


In [6]:
from dezero import Variable

x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1.grad)

Variable([11 12 13])
Variable([3])
